# 食品图像识别与营养成分分析系统

本项目实现了一个基于深度学习的食品图像识别系统，能够从图像中识别食品类型，并提供相应的营养成分信息。

## 项目流程
1. 数据准备与预处理
2. 营养数据清洗与映射
3. 模型训练与优化（ResNet-18）
4. 模型评估与比较
5. 系统集成

## 参考文献

- FoodAI: https://arxiv.org/abs/2004.13385
- Nutritionix: https://www.nutritionix.com/
- DeepFood: https://arxiv.org/abs/1606.05675
- Vision Transformers (ViT): https://arxiv.org/abs/2010.11929
- Food Recognition Dataset: https://www.kaggle.com/datasets/sainikhileshreddy/food-recognition-2022/data
- USDA Food Composition Database: https://www.ars.usda.gov/northeast-area/beltsville-md-bhnrc/beltsville-human-nutrition-research-center/food-surveys-research-group/docs/fndds-download-databases/
- COFID Dataset: https://www.gov.uk/government/publications/composition-of-foods-integrated-dataset-cofid
- Food Recognition Dataset (Dataset Ninja): https://datasetninja.com/food-recognition

## 0. 数据下载与准备

首先，我们需要下载并准备所需的数据集。下面的代码将帮助我们获取所有必要的数据源。

In [2]:
# 安装必要的依赖
%pip install gdown kaggle tqdm requests pandas numpy matplotlib opencv-python pillow tensorflow scikit-learn seaborn

# 导入所需的库
import os
import requests
import zipfile
import gdown
import kaggle
from tqdm import tqdm


  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
   ---------------------------------------- 0.0/375.9 MB ? eta -:--:--
   ---------------------------------------- 3.4/375.9 MB 16.7 MB/s eta 0:00:23
    --------------------------------------- 7.1/375.9 MB 16.7 MB/s eta 0:00:23
   - -------------------------------------- 10.7/375.9 MB 16.8 MB/s eta 0:00:22
   - -------------------------------------- 14.4/375.9 MB 16.8 MB/s eta 0:00:22
   - -------------------------------------- 18.1/375.9 MB 17.3 MB/s eta 0:00:21
   -- ------------------------------------- 21.8/375.9 MB 17.0 MB/s eta 0:00:21
   -- ------------------------------------- 24.6/375.9 MB 16.6 MB/s eta 0:00:22
   -- ------------------------------------- 27.8/375.9 MB 16.3 MB/s eta

OSError: Could not find kaggle.json. Make sure it's located in C:\Users\James\.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/

In [ ]:
# 创建数据下载目录
download_dir = os.path.join('d:\\AAmachine learning\\Group Project', 'downloaded_datasets')
os.makedirs(download_dir, exist_ok=True)

# 定义各数据集下载函数
def download_kaggle_dataset(dataset_name, path):
    """
    下载Kaggle数据集
    注意：需要提前设置Kaggle API凭证 (.kaggle/kaggle.json)
    """
    try:
        print(f"下载 {dataset_name} 数据集...")
        # 注意：如需使用Kaggle API，请先配置kaggle.json文件
        # kaggle.api.authenticate()
        # kaggle.api.dataset_download_files(dataset_name, path=path, unzip=True)
        print(f"请手动从 https://www.kaggle.com/datasets/{dataset_name} 下载数据集到 {path} 目录")
        print("然后解压到同一目录")
    except Exception as e:
        print(f"下载Kaggle数据集时出错: {e}")
        print("请手动下载数据集")

def download_usda_database(path):
    """
    下载USDA食品成分数据库
    """
    usda_url = "https://www.ars.usda.gov/ARSUserFiles/80400525/Data/FNDDS/FNDDS_2017_2018.zip"
    download_file(usda_url, os.path.join(path, "FNDDS_2017_2018.zip"))

def download_cofid_dataset(path):
    """
    下载COFID数据集
    """
    cofid_url = "https://assets.publishing.service.gov.uk/media/5c6da48e40f0b647b35c7904/McCance_Widdowsons_Composition_of_Foods_Integrated_Dataset_2019.xlsx"
    download_file(cofid_url, os.path.join(path, "COFID_2019.xlsx"))

def download_dataset_ninja(path):
    """
    下载Dataset Ninja食品识别数据集
    """
    print("请手动从 https://datasetninja.com/food-recognition 下载数据集到指定目录")

def download_file(url, destination):
    """
    从URL下载文件，显示进度条
    """
    try:
        response = requests.get(url, stream=True)
        total_size = int(response.headers.get('content-length', 0))
        block_size = 1024
        progress_bar = tqdm(total=total_size, unit='iB', unit_scale=True)
        
        with open(destination, 'wb') as f:
            for data in response.iter_content(block_size):
                progress_bar.update(len(data))
                f.write(data)
        progress_bar.close()
        
        if total_size != 0 and progress_bar.n != total_size:
            print("下载不完整")
        else:
            print(f"成功下载到 {destination}")
    except Exception as e:
        print(f"下载文件时出错: {e}")
        print("请手动下载文件")

In [ ]:
# 下载食品识别数据集
kaggle_dataset_path = os.path.join(download_dir, 'food_recognition')
os.makedirs(kaggle_dataset_path, exist_ok=True)
download_kaggle_dataset("sainikhileshreddy/food-recognition-2022", kaggle_dataset_path)

# 下载营养数据
nutrition_data_path = os.path.join(download_dir, 'nutrition_data')
os.makedirs(nutrition_data_path, exist_ok=True)
download_usda_database(nutrition_data_path)
download_cofid_dataset(nutrition_data_path)

# 下载Dataset Ninja食品识别数据
ninja_dataset_path = os.path.join(download_dir, 'dataset_ninja')
os.makedirs(ninja_dataset_path, exist_ok=True)
download_dataset_ninja(ninja_dataset_path)

In [ ]:
# 解压下载的ZIP文件
def extract_all_zips(directory):
    """
    解压目录中的所有ZIP文件
    """
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.zip'):
                zip_path = os.path.join(root, file)
                extract_dir = os.path.splitext(zip_path)[0]
                try:
                    print(f"解压 {zip_path} 到 {extract_dir}...")
                    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                        zip_ref.extractall(extract_dir)
                    print(f"解压完成: {file}")
                except Exception as e:
                    print(f"解压 {file} 时出错: {e}")

# 解压所有下载的ZIP文件
extract_all_zips(download_dir)

## 1. 数据探索与整合

现在，我们将探索和整合下载的数据集，创建一个统一的数据结构。

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import shutil
from collections import Counter

# 定义基本目录
base_dir = r'D:\AAmachine learning\Group Project\Datasets'
download_dir = r'D:\AAmachine learning\Group Project\downloaded_datasets'
integrated_data_dir = r'D:\AAmachine learning\Group Project\integrated_data'

# 创建整合数据目录
os.makedirs(integrated_data_dir, exist_ok=True)
os.makedirs(os.path.join(integrated_data_dir, 'images'), exist_ok=True)
os.makedirs(os.path.join(integrated_data_dir, 'annotations'), exist_ok=True)
os.makedirs(os.path.join(integrated_data_dir, 'nutrition'), exist_ok=True)

In [ ]:
# 探索现有数据集结构
def explore_dataset_structure(directory):
    """
    探索目录结构并返回文件统计信息
    """
    stats = {'directories': [], 'files': [], 'extensions': []}
    for root, dirs, files in os.walk(directory):
        rel_path = os.path.relpath(root, directory)
        if rel_path != '.':
            stats['directories'].append(rel_path)
        
        for file in files:
            stats['files'].append(os.path.join(rel_path, file))
            _, ext = os.path.splitext(file)
            stats['extensions'].append(ext)
    
    # 统计文件类型分布
    ext_counts = Counter(stats['extensions'])
    
    # 输出汇总信息
    print(f"目录总数: {len(stats['directories'])}")
    print(f"文件总数: {len(stats['files'])}")
    print("\n文件类型分布:")
    for ext, count in ext_counts.most_common():
        print(f"{ext}: {count}")
    
    return stats

# 探索现有数据集
print("原始数据集结构:")
original_stats = explore_dataset_structure(base_dir)

# 如果下载数据集已存在，也探索它的结构
if os.path.exists(download_dir) and os.listdir(download_dir):
    print("\n下载的数据集结构:")
    download_stats = explore_dataset_structure(download_dir)

## 2. 图像预处理与数据增强

接下来，我们将实现图像预处理和数据增强功能，统一图像大小为224x224像素，并应用数据增强技术。

In [ ]:
import cv2
from PIL import Image, ImageEnhance, ImageOps
import random
import shutil

def standardize_image(image_path, target_size=(224, 224), output_path=None):
    """
    读取图像并统一大小为224x224像素，RGB色彩空间
    """
    try:
        # 使用OpenCV读取图像
        img = cv2.imread(image_path)
        if img is None:
            return None
        
        # 转换为RGB（OpenCV默认是BGR）
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # 调整大小，保持长宽比
        h, w = img.shape[:2]
        if h > w:
            new_h, new_w = target_size[0] * h // w, target_size[0]
        else:
            new_h, new_w = target_size[1], target_size[1] * w // h
        
        img = cv2.resize(img, (new_w, new_h))
        
        # 居中裁剪到目标尺寸
        h, w = img.shape[:2]
        start_x = (w - target_size[0]) // 2
        start_y = (h - target_size[1]) // 2
        img = img[start_y:start_y+target_size[1], start_x:start_x+target_size[0]]
        
        # 如果有输出路径，则保存图像
        if output_path:
            cv2.imwrite(output_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
        
        return img
    except Exception as e:
        print(f"处理图像 {image_path} 时出错: {e}")
        return None

def apply_augmentation(img, augmentation_type=None):
    """
    应用指定类型的数据增强
    """
    # 转换为PIL图像
    pil_img = Image.fromarray(img)
    
    # 如果未指定增强类型，则随机选择
    if augmentation_type is None:
        augmentation_type = random.choice(['rotate', 'flip', 'brightness', 'contrast', 'none'])
    
    if augmentation_type == 'rotate':
        # 随机旋转 -30 到 30 度
        angle = random.uniform(-30, 30)
        pil_img = pil_img.rotate(angle, resample=Image.BICUBIC, expand=False)
    elif augmentation_type == 'flip':
        # 水平翻转
        pil_img = ImageOps.mirror(pil_img)
    elif augmentation_type == 'brightness':
        # 调整亮度，因子0.7-1.3
        factor = random.uniform(0.7, 1.3)
        enhancer = ImageEnhance.Brightness(pil_img)
        pil_img = enhancer.enhance(factor)
    elif augmentation_type == 'contrast':
        # 调整对比度，因子0.7-1.3
        factor = random.uniform(0.7, 1.3)
        enhancer = ImageEnhance.Contrast(pil_img)
        pil_img = enhancer.enhance(factor)
    
    # 转换回numpy数组
    return np.array(pil_img)

def preprocess_and_augment_dataset(input_dir, output_dir, augmentation_count=1):
    """
    预处理并增强数据集
    input_dir: 输入图像目录
    output_dir: 输出图像目录
    augmentation_count: 每张图像生成的增强图像数量
    """
    os.makedirs(output_dir, exist_ok=True)
    
    image_files = [f for f in os.listdir(input_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
    
    print(f"开始处理 {len(image_files)} 张图像...")
    for i, img_file in enumerate(tqdm(image_files)):
        # 原始图像路径
        img_path = os.path.join(input_dir, img_file)
        
        # 预处理原始图像
        output_path = os.path.join(output_dir, img_file)
        img = standardize_image(img_path, output_path=output_path)
        
        if img is not None:
            # 应用数据增强
            for j in range(augmentation_count):
                # 选择一种增强方法
                aug_type = random.choice(['rotate', 'flip', 'brightness', 'contrast'])
                
                # 应用增强
                aug_img = apply_augmentation(img, aug_type)
                
                # 生成增强图像的文件名
                base_name, ext = os.path.splitext(img_file)
                aug_file = f"{base_name}_aug{j+1}_{aug_type}{ext}"
                
                # 保存增强图像
                aug_path = os.path.join(output_dir, aug_file)
                Image.fromarray(aug_img).save(aug_path)
        
        # 显示进度
        if (i + 1) % 100 == 0 or i + 1 == len(image_files):
            print(f"已处理 {i + 1}/{len(image_files)} 张图像")
    
    print(f"图像预处理与增强完成，结果保存到 {output_dir}")

## 3. 营养数据清洗与映射

接下来，我们将清洗和整合营养数据，创建食品名称到营养成分的映射。

In [ ]:
import pandas as pd
import numpy as np
import json
import sqlite3
import os
from fuzzywuzzy import process, fuzz

def load_usda_data(directory):
    """
    加载USDA营养数据
    """
    try:
        # FNDDS通常包含多个CSV文件，我们需要找到并加载食品和营养成分相关的表
        # 示例文件路径，实际路径可能需要调整
        food_file = os.path.join(directory, 'FNDDS_2017_2018', 'food.csv')
        nutrient_file = os.path.join(directory, 'FNDDS_2017_2018', 'nutrient.csv')
        
        if os.path.exists(food_file) and os.path.exists(nutrient_file):
            food_df = pd.read_csv(food_file)
            nutrient_df = pd.read_csv(nutrient_file)
            print(f"成功加载USDA数据: {len(food_df)} 种食品, {len(nutrient_df)} 条营养记录")
            return food_df, nutrient_df
        else:
            print("无法找到USDA数据文件，请检查下载是否成功或目录结构是否正确")
            # 创建一个示例结构的空DataFrame
            food_df = pd.DataFrame(columns=['food_code', 'food_description'])
            nutrient_df = pd.DataFrame(columns=['food_code', 'nutrient_code', 'nutrient_value'])
            return food_df, nutrient_df
    except Exception as e:
        print(f"加载USDA数据时出错: {e}")
        # 创建一个示例结构的空DataFrame
        food_df = pd.DataFrame(columns=['food_code', 'food_description'])
        nutrient_df = pd.DataFrame(columns=['food_code', 'nutrient_code', 'nutrient_value'])
        return food_df, nutrient_df

def load_cofid_data(file_path):
    """
    加载COFID营养数据
    """
    try:
        if os.path.exists(file_path):
            cofid_df = pd.read_excel(file_path)
            print(f"成功加载COFID数据: {len(cofid_df)} 条记录")
            return cofid_df
        else:
            print("无法找到COFID数据文件，请检查下载是否成功")
            # 创建一个示例结构的空DataFrame
            return pd.DataFrame(columns=['Food Code', 'Food Name', 'Energy (kcal)'])
    except Exception as e:
        print(f"加载COFID数据时出错: {e}")
        # 创建一个示例结构的空DataFrame
        return pd.DataFrame(columns=['Food Code', 'Food Name', 'Energy (kcal)'])

def clean_and_standardize_nutrition_data(usda_food, usda_nutrient, cofid_df):
    """
    清洗和标准化营养数据
    """
    # 创建标准化的营养数据框架
    columns = ['food_name', 'calories', 'protein_g', 'carbs_g', 'fat_g', 'fiber_g', 'sugar_g', 'source']
    nutrition_df = pd.DataFrame(columns=columns)
    
    # 处理USDA数据
    if not usda_food.empty and not usda_nutrient.empty:
        try:
            # 这里需要按照实际的USDA数据结构调整代码
            # 以下是一个示例处理流程
            # 合并食品和营养成分数据
            merged_df = pd.merge(usda_food, usda_nutrient, on='food_code')
            
            # 从营养素代码映射到标准字段名
            # 这里需要根据实际的USDA数据结构确定营养素代码
            nutrient_mapping = {
                '208': 'calories',  # 假设208是热量的代码
                '203': 'protein_g', # 假设203是蛋白质的代码
                '205': 'carbs_g',   # 假设205是碳水化合物的代码
                '204': 'fat_g',     # 假设204是脂肪的代码
                '291': 'fiber_g',   # 假设291是纤维的代码
                '269': 'sugar_g'    # 假设269是糖的代码
            }
            
            # 将营养数据透视为宽格式
            pivoted = merged_df.pivot_table(index='food_description', 
                                           columns='nutrient_code', 
                                           values='nutrient_value',
                                           aggfunc='mean').reset_index()
            
            # 重命名列
            pivoted.rename(columns={'food_description': 'food_name'}, inplace=True)
            for code, name in nutrient_mapping.items():
                if code in pivoted.columns:
                    pivoted.rename(columns={code: name}, inplace=True)
            
            # 添加来源列
            pivoted['source'] = 'USDA'
            
            # 保留需要的列
            usda_nutrition = pivoted[['food_name'] + [col for col in columns[1:-1] if col in pivoted.columns] + ['source']]
            
            # 追加到主营养数据框架
            nutrition_df = pd.concat([nutrition_df, usda_nutrition], ignore_index=True)
            
        except Exception as e:
            print(f"处理USDA数据时出错: {e}")
    
    # 处理COFID数据
    if not cofid_df.empty:
        try:
            # 根据实际的COFID数据结构调整代码
            # 创建一个新的DataFrame来存储从COFID提取的数据
            cofid_nutrition = pd.DataFrame()
            
            # 映射列名
            cofid_nutrition['food_name'] = cofid_df['Food Name']
            cofid_nutrition['calories'] = cofid_df['Energy (kcal)']
            
            # 根据实际数据结构添加其他营养素
            # 例如：cofid_nutrition['protein_g'] = cofid_df['Protein (g)'] 
            
            # 添加来源列
            cofid_nutrition['source'] = 'COFID'
            
            # 追加到主营养数据框架
            nutrition_df = pd.concat([nutrition_df, cofid_nutrition], ignore_index=True)
            
        except Exception as e:
            print(f"处理COFID数据时出错: {e}")
    
    # 清洗数据
    # 删除重复的食品名称（保留第一次出现的）
    nutrition_df.drop_duplicates(subset='food_name', keep='first', inplace=True)
    
    # 处理缺失值
    numeric_cols = ['calories', 'protein_g', 'carbs_g', 'fat_g', 'fiber_g', 'sugar_g']
    for col in numeric_cols:
        if col in nutrition_df.columns:
            # 用0填充缺失值
            nutrition_df[col] = nutrition_df[col].fillna(0)
    
    # 标准化食品名称
    nutrition_df['food_name'] = nutrition_df['food_name'].str.lower().str.strip()
    
    # 不足的列填充为0
    for col in columns[1:-1]:
        if col not in nutrition_df.columns:
            nutrition_df[col] = 0
    
    return nutrition_df

def create_nutrition_database(nutrition_df, db_path):
    """
    创建SQLite营养数据库
    """
    conn = sqlite3.connect(db_path)
    
    # 将数据框写入SQLite表
    nutrition_df.to_sql('food_nutrition', conn, if_exists='replace', index=False)
    
    # 创建索引以加快查询
    conn.execute('CREATE INDEX IF NOT EXISTS idx_food_name ON food_nutrition(food_name)')
    
    # 提交更改
    conn.commit()
    conn.close()
    
    print(f"营养数据库创建成功: {db_path}")
    return db_path

def export_nutrition_to_json(db_path, json_path):
    """
    将营养数据从SQLite导出为JSON
    """
    # 连接数据库
    conn = sqlite3.connect(db_path)
    
    # 查询所有数据
    nutrition_df = pd.read_sql('SELECT * FROM food_nutrition', conn)
    
    # 将数据框转换为字典
    nutrition_dict = {}
    for _, row in nutrition_df.iterrows():
        food_name = row.pop('food_name')
        nutrition_dict[food_name] = row.to_dict()
    
    # 写入JSON文件
    with open(json_path, 'w') as f:
        json.dump(nutrition_dict, f, indent=4)
    
    conn.close()
    
    print(f"营养数据已导出为JSON: {json_path}")
    return nutrition_dict

def create_food_nutrition_mapping(food_classes, nutrition_df, output_path):
    """
    创建食品类别到营养信息的映射
    """
    # 创建映射字典
    mapping = {}
    
    for food_class in food_classes:
        food_name = food_class['title'].lower()
        
        # 尝试直接匹配
        match = nutrition_df[nutrition_df['food_name'] == food_name]
        
        # 如果没有直接匹配，尝试模糊匹配
        if match.empty:
            # 使用模糊匹配找到最接近的食品名称
            food_names = nutrition_df['food_name'].tolist()
            best_match = process.extractOne(food_name, food_names, scorer=fuzz.token_sort_ratio)
            
            if best_match and best_match[1] > 70:  # 匹配度阈值
                match_name = best_match[0]
                match = nutrition_df[nutrition_df['food_name'] == match_name]
        
        # 如果找到了匹配，添加到映射中
        if not match.empty:
            nutrition_info = match.iloc[0].to_dict()
            del nutrition_info['food_name']  # 移除多余的食品名称字段
            mapping[food_name] = nutrition_info
        else:
            # 如果没有找到匹配，使用默认值
            mapping[food_name] = {
                'calories': 0,
                'protein_g': 0,
                'carbs_g': 0,
                'fat_g': 0,
                'fiber_g': 0,
                'sugar_g': 0,
                'source': 'default'
            }
    
    # 保存映射到文件
    with open(output_path, 'w') as f:
        json.dump(mapping, f, indent=4)
    
    print(f"食品-营养成分映射已创建: {output_path}")
    return mapping

## 4. 模型训练与优化 - ResNet-18

接下来，我们将实现并训练ResNet-18模型用于食品识别

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

def build_resnet18_model(input_shape=(224, 224, 3), num_classes=1000):
    """构建ResNet-18模型"""
    # 注：Keras没有直接提供ResNet18，我们这里用ResNet50作为示例
    # 在实际项目中，你可以导入PyTorch中的ResNet18或手动构建
    base_model = ResNet50(weights='imagenet', 
                         include_top=False, 
                         input_shape=input_shape)
    
    # 冻结基础模型的卷积层
    for layer in base_model.layers:
        layer.trainable = False
    
    # 添加自定义分类层
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    
    # 最终模型
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # 编译模型
    model.compile(optimizer=Adam(learning_rate=1e-4),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

def train_model(model, X_train, y_train, X_val, y_val, batch_size=32, epochs=20):
    """训练模型"""
    # 定义回调
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3),
        ModelCheckpoint('best_resnet_food_model.h5', save_best_only=True)
    ]
    
    # 数据增强器
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True
    )
    
    # 训练模型
    history = model.fit(
        datagen.flow(X_train, y_train, batch_size=batch_size),
        steps_per_epoch=len(X_train) // batch_size,
        epochs=epochs,
        validation_data=(X_val, y_val),
        callbacks=callbacks
    )
    
    return history

def evaluate_model(model, X_test, y_test):
    """评估模型"""
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f"测试集损失: {loss:.4f}")
    print(f"测试集准确率: {accuracy:.4f}")
    
    # 预测类别
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    return y_pred_classes

def plot_training_history(history):
    """绘制训练历史"""
    plt.figure(figsize=(12, 4))
    
    # 绘制准确率
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='训练准确率')
    plt.plot(history.history['val_accuracy'], label='验证准确率')
    plt.title('模型准确率')
    plt.xlabel('Epoch')
    plt.ylabel('准确率')
    plt.legend()
    
    # 绘制损失
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='训练损失')
    plt.plot(history.history['val_loss'], label='验证损失')
    plt.title('模型损失')
    plt.xlabel('Epoch')
    plt.ylabel('损失')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

In [ ]:
# 注意：由于计算资源和时间限制，这里我们只加载小样本数据集进行训练演示
# 在实际项目中，你应该使用完整的数据集

# 加载训练数据（小样本）
print("加载训练数据...")
X_train_full, y_train_full, class_mapping = load_and_preprocess_dataset(dataset_type='training', max_samples=100, augment=True)

if len(X_train_full) > 0:
    # 划分训练集和验证集
    X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)
    
    # 加载少量测试数据
    print("加载测试数据...")
    X_test, y_test, _ = load_and_preprocess_dataset(dataset_type='test', max_samples=50)
    
    # 获取类别数量
    num_classes = len(set(y_train_full))
    print(f"类别数量: {num_classes}")
    
    # 构建模型
    print("构建ResNet模型...")
    model = build_resnet18_model(input_shape=(224, 224, 3), num_classes=num_classes)
    print(model.summary())
    
    # 训练模型（仅几个epoch用于演示）
    print("训练模型...")
    history = train_model(model, X_train, y_train, X_val, y_val, batch_size=16, epochs=5)
    
    # 绘制训练历史
    plot_training_history(history)
    
    # 评估模型
    print("评估模型...")
    y_pred_classes = evaluate_model(model, X_test, y_test)
else:
    print("没有足够的训练数据，跳过模型训练。")

## 5. 模型评估与性能分析

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import seaborn as sns

def detailed_evaluation(y_true, y_pred, class_mapping):
    """详细评估模型性能"""
    # 反转类别映射
    id_to_class = {v: k for k, v in class_mapping.items()}
    
    # 计算基础指标
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    
    print(f"准确率: {accuracy:.4f}")
    print(f"精确率: {precision:.4f}")
    print(f"召回率: {recall:.4f}")
    print(f"F1分数: {f1:.4f}")
    
    # 生成分类报告
    target_names = [id_to_class[i] for i in sorted(set(y_true))]
    report = classification_report(y_true, y_pred, target_names=target_names)
    print("\n分类报告:")
    print(report)
    
    # 生成混淆矩阵
    cm = confusion_matrix(y_true, y_pred)
    
    # 绘制混淆矩阵热图
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=[id_to_class[i][:15] for i in range(len(set(y_true)))],
                yticklabels=[id_to_class[i][:15] for i in range(len(set(y_true)))])
    plt.title('混淆矩阵')
    plt.xlabel('预测标签')
    plt.ylabel('真实标签')
    plt.xticks(rotation=90)
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()
    
    return accuracy, precision, recall, f1

In [ ]:
# 如果有测试数据和预测结果，进行详细评估
if 'y_test' in locals() and 'y_pred_classes' in locals() and len(y_test) > 0:
    print("执行详细的模型评估...")
    metrics = detailed_evaluation(y_test, y_pred_classes, class_mapping)
else:
    print("没有测试数据或预测结果，跳过详细评估。")

## 6. 营养预测系统集成

In [ ]:
def predict_food_and_nutrition(model, image_path, class_mapping, nutrition_data, confidence_threshold=0.7):
    """预测食品类型并返回营养信息"""
    # 预处理图像
    img = preprocess_image(image_path)
    if img is None:
        return {"error": "无法处理图像"}
    
    # 调整形状以适应模型输入
    img_batch = np.expand_dims(img, axis=0)
    
    # 预测
    predictions = model.predict(img_batch)[0]
    top_class_idx = np.argmax(predictions)
    confidence = predictions[top_class_idx]
    
    # 反转类别映射
    id_to_class = {v: k for k, v in class_mapping.items()}
    predicted_class = id_to_class.get(top_class_idx, "未知")
    
    # 检查置信度是否满足阈值
    result = {
        "food_name": predicted_class,
        "confidence": float(confidence)
    }
    
    if confidence >= confidence_threshold:
        # 获取营养信息
        if predicted_class in nutrition_data:
            result["nutrition"] = nutrition_data[predicted_class]
        else:
            result["nutrition"] = {"error": "没有可用的营养信息"}
    else:
        result["message"] = "置信度低于阈值，无法可靠识别"
    
    return result

In [ ]:
# 演示一个完整的预测过程
# 首先找到一个测试图像
test_img_dir = os.path.join(base_dir, 'test', 'img')
test_images = [f for f in os.listdir(test_img_dir) if f.endswith('.jpg')][:5]

if test_images and 'model' in locals():
    for test_img in test_images:
        # 完整路径
        img_path = os.path.join(test_img_dir, test_img)
        
        # 预测食品和营养成分
        result = predict_food_and_nutrition(model, img_path, class_mapping, nutrition_data)
        
        # 显示图像
        plt.figure(figsize=(8, 6))
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(img)
        
        # 显示预测结果
        plt.title(f"预测: {result['food_name']} (置信度: {result['confidence']:.2f})")
        
        # 显示营养信息
        info_text = ""
        if "nutrition" in result:
            nutrition = result["nutrition"]
            info_text = f"热量: {nutrition.get('calories', 'N/A')} kcal\n"
            info_text += f"蛋白质: {nutrition.get('protein_g', 'N/A')} g\n"
            info_text += f"碳水化合物: {nutrition.get('carbs_g', 'N/A')} g\n"
            info_text += f"脂肪: {nutrition.get('fat_g', 'N/A')} g"
        else:
            info_text = result.get("message", "无营养信息")
        
        plt.figtext(0.5, -0.1, info_text, ha="center", fontsize=12, bbox={"facecolor":"white", "alpha":0.8, "pad":5})
        plt.tight_layout()
        plt.show()

## 7. API设计（Flask应用示例）

以下是如何实现一个Flask API来提供食品识别和营养分析服务的示例代码。在实际项目中，你可以将这段代码保存到单独的Python文件中运行。

In [ ]:
# Flask API示例代码
'''
from flask import Flask, request, jsonify
import numpy as np
import cv2
import json
import os
from tensorflow.keras.models import load_model

app = Flask(__name__)

# 加载模型和必要的数据
model = load_model('best_resnet_food_model.h5')

# 加载类别映射
with open('class_mapping.json', 'r') as f:
    class_mapping = json.load(f)

# 加载营养数据
with open('food_nutrition.json', 'r') as f:
    nutrition_data = json.load(f)

def preprocess_image(image, target_size=(224, 224)):
    """预处理图像"""
    # 调整大小
    img_resized = cv2.resize(image, target_size)
    # 转换为RGB
    img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)
    # 归一化
    img_normalized = img_rgb / 255.0
    return img_normalized

@app.route('/predict', methods=['POST'])
def predict():
    """接收图像并返回食物识别和营养信息"""
    # 检查是否收到图像
    if 'image' not in request.files:
        return jsonify({'error': '没有接收到图像'}), 400
        
    # 读取图像
    file = request.files['image']
    img = cv2.imdecode(np.frombuffer(file.read(), np.uint8), cv2.IMREAD_COLOR)
    
    # 定义置信度阈值 (可通过请求参数配置)
    confidence_threshold = float(request.args.get('threshold', 0.7))
    
    # 预处理图像
    processed_img = preprocess_image(img)
    img_batch = np.expand_dims(processed_img, axis=0)
    
    # 预测
    predictions = model.predict(img_batch)[0]
    top_class_idx = np.argmax(predictions)
    confidence = float(predictions[top_class_idx])
    
    # 反转类别映射
    id_to_class = {int(v): k for k, v in class_mapping.items()}
    predicted_class = id_to_class.get(top_class_idx, "未知")
    
    # 准备响应
    response = {
        "food_name": predicted_class,
        "confidence": confidence
    }
    
    # 如果置信度超过阈值，添加营养信息
    if confidence >= confidence_threshold:
        if predicted_class in nutrition_data:
            response["nutrition"] = nutrition_data[predicted_class]
        else:
            response["message"] = "没有可用的营养信息"
    else:
        response["message"] = "置信度低于阈值，无法可靠识别"
    
    return jsonify(response)

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=5000)
'''

## 8. 总结与未来工作

在这个项目中，我们实现了一个完整的食品图像识别和营养分析系统，包括：

1. 图像预处理和数据增强
2. 营养数据库的创建与映射
3. ResNet-18模型的训练与优化
4. 模型评估与性能分析
5. 预测系统的集成
6. API设计（Flask示例）

### 未来工作

1. **模型改进**：尝试其他深度学习架构如EfficientNet、ViT等，并进行集成学习
2. **多标签分类**：支持识别包含多种食物的图像
3. **精确营养估计**：基于食物体积/重量等因素提供更精确的营养信息
4. **用户界面**：开发移动应用或Web界面，提供更好的用户体验
5. **特定饮食建议**：基于用户的健康状况和饮食习惯提供个性化建议